<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2018


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>

<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> First-level analysis </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b> November 19, 2018</b></p>

<hr style='height:5px;border:none' />

# First-level analysis - theoretical background
<hr style="height:1px;border:none" />

The first-level analysis refers to a statistical analysis of fMRI data on a single subject (of a single session and/or a single run). The main goal in the first-level analysis is to fit a temporal model to the fMRI data to quantify the activation of interest as contrast(s). 

## Predicted BOLD signals
Say, your fMRI experiement consists of epochs of baseline and stimulus.

<img style='width: 400px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_ExperimentDesign.png?raw=true' alt='Experiment design'/>

In this experiment, you anticipate activations associated with the stimulus, and you try to capture that with BOLD (blood oxygenation-level dependent) fMRI. However, after a neural activity, you can only observe delayed response in the resultant BOLD signal due to a hemodynamic delay. Such a delay is modeled by a function known as a **hemodynamic response function (HRF)**. The predicted BOLD signal, from an fMRI experiment, is the underlying neural activity convolved by an HRF.

<img style='width: 700px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_BOLDSignal.png?raw=true' alt='Predicted BOLD signal'/>

In most fMRI analysis software tools, you can generate boxcar signals for predicted neural activities simply by providing event onset times and durations. There are also a number of HRF models available for you to choose from. Your predicted neural response is convolved by the HRF of your choice, so you don't have to generate predicted BOLD fMRI signals by yourself.

## High-pass filtering

During an fMRI experiment, BOLD fMRI signals often fluctuates slowly, the phenomenon known as the **low-frequency drift**. 

<img style='width: 400px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_LowFreqDrift.png?raw=true' alt='Low frequency drift'/>

This poses a challenge in statistical analyses of the data. Luckily low-frequency drift can be easily removed from fMRI data. Since it is low frequency, a high-pass filter can eliminate it. 

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_HighPass.png?raw=true' alt='High-pass filter before and after'/>

A cut-off frequency of around 0.01Hz (or a period of 100s) seems to work well for typical fMRI data. 


## GLM

GLM stands for **general linear model**. It is a statistical model that can be used for different types of analyses:
  * ANOVA (analysis of variance): Comparison of group means
  * ANCOVA (analysis of covariance): ANOVA adjusted for continuous variables
  * Linear regression, simple & multiple
  * T-test: Statistical test on mean(s)
  * F-test: Statistical test to compare variances
  
<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_GLMSimple.png?raw=true' alt='GLM for simple regression'/>

For your fMRI experiment data, you can fit such a model at each voxel separately.

<img style='width: 400px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_GLMSimple_fMRI.png?raw=true' alt='GLM for simple regression, fMRI'/>

In this case, $\beta$ parameters from all voxels collectively form a 3D image.

In a more realistic case, there are more than one regressors in a GLM. 

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_GLMMultiple.png?raw=true' alt='GLM for multiple regression'/>

Or, using a matrix notation, one can describe this model with vectors:

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_GLMMultipleMatrix.png?raw=true' alt='GLM for multiple regression, matrix notation'/>

In an fMRI experiment, a GLM with multiple regressors will result in multiple beta images.

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_GLMMultiple_fMRI.png?raw=true' alt='GLM for multiple regression, fMRI experiment'/>

### Notes on GLM for fMRI
***fMRI time series is temporally correlated***
   * Not independent data points. Violation in the assumptions of an ordinary regression model
   
***The temporal correlation needs to be addressed somehow***
   * **Pre-whitenening**: Temporal correlation is minimized so that data points are almost independent. 
   * **Explicitly modeling temporal correlation**: The temporal correlation between neighboring time points is estimated, and used in the statistical model. 

***HRF is different in different parts of the brain***
   * **Temporal derivatives** of the predicted BOLD signal can be included also included as regressors in the GLM
   * Inclusion of temporal derivatives can correct delays in BOLD signals

***Second line of difference against subject motion***
   * The **rigid-body transformation parameters** from the motion correction step can be included as regressors in the GLM
   * Any motion-associated fluctuation in BOLD fMRI can be corrected.

   
## Contrasts

A contrast is a linear combination of regression parameters $\beta$s. For example,

  * $\beta_1$: Activation associated with condition 1
  * $-\beta_2$: Deactivation associated with condition 2
  * $\beta_2-\beta_1$: Activation difference between conditions 2 and 1. (Condition 2 > Condition 1)
  * $\beta_3-\beta_1$: Activation difference between conditions 3 and 1. (Condition 3 > Condition 1)

Or, using a vector notation, we can describe these contrasts as

$$
    \beta_1 = \begin{bmatrix}1 & 0 & 0 \end{bmatrix}
        \begin{bmatrix}
           \beta_{1} \\
           \beta_{2} \\
           \beta_{3}
         \end{bmatrix}
$$

$$
    -\beta_2 = \begin{bmatrix}0 & -1 & 0 \end{bmatrix}
        \begin{bmatrix}
           \beta_{1} \\
           \beta_{2} \\
           \beta_{3}
         \end{bmatrix}
$$

$$
    \beta_2-\beta_1 = \begin{bmatrix}-1 & 1 & 0 \end{bmatrix}
        \begin{bmatrix}
           \beta_{1} \\
           \beta_{2} \\
           \beta_{3}
         \end{bmatrix}
$$

$$
    \beta_1+\beta_2+\beta_3 = \begin{bmatrix}1 & 1 & 1 \end{bmatrix}
        \begin{bmatrix}
           \beta_{1} \\
           \beta_{2} \\
           \beta_{3}
         \end{bmatrix}
$$

In a first-level analysis, a contrast image can be calculated from beta-images. For example, then contrast $\beta_2 - \beta_1$ can be calculated as:

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_ContrastImage.png?raw=true' alt='Contrast image example'/>

Once the contrast image is generated, it is possible to perform a T-test (at each voxel separately) by calculating a T-statistic image. A T-statistic image can be calculated by dividing a contrast image by the residual standard error (SE) image, with an appropriate normalizing constant.

<img style='width: 500px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Level1_TStatImage.png?raw=true' alt='T-statistic image calculation'/>

In fMRI data analysis, the directionality matters. Highly positive T-statistics are associated with activations or effects described in the contrast. Highly negative T-statistics, on the other hand, represent deactivations or  negative effects.

### Omnibus testing with an F-contrast

Say, you are interested in the difference among any of 3 conditions. In other words, you want to examine:
  * $\beta_3 - \beta_2$ (contrast vector [0 -1 1])
  * $\beta_3 - \beta_1$ (contrast vector [-1 0 1])
  * $\beta_2 - \beta_1$ (contrast vector [-1 1 0])

You can examine each of these separately by a t-test. Or, you can examine these together collectively using an **F-contrast**. An F-contrast can be written as a matrix. In this particular example, the contrast is described by stacking the contrasts into a matrix.

$$
        \begin{bmatrix}
           0 & -1 & 1 \\
           -1 & 0 & 1 \\
           -1 & 1 & 0 
         \end{bmatrix}
$$

If you are interested in an F-contrast, an F-contrast image as well as an F-statistic image can be calculated (details not shown). An F-statistic is non-directional. Thus a significantly large F-statistic may mean either activation or deactivation (or positive or negative differences), but the directionality cannot be determined by an F-statistic alone. Moreover, it is hard to determine which comparison contributed to the significant F-test outcome.

# First-level analysis - FSL approach
<hr style="height:1px;border:none" />

# First-level analysis - SPM approach
<hr style="height:1px;border:none" />

